In [ ]:
import os
import sys
current = os.path.dirname(os.path.realpath("Visualize_Pairwise.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent)
from transformers import CLIPProcessor, CLIPModel
import torch
import torch.hub
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import math

from match_utils import matching, stats, proggan, nethook, dataset, loading, plotting, layers, models, visualize_pairwisematch

In [ ]:
device = torch.device('cuda:3')

In [ ]:
gan, gan_layers = models.load_gan("stylegan3", device)
discr, discr_layers = models.load_discr("dino", device)

In [ ]:
ganlayers, discrlayers = layers.get_layers(gan,gan_layers, discr, discr_layers,"stylegan3", "dino", device)

In [ ]:
table, gan_stats, discr_stats = loading.load_stats("matches/stylegan3_dino_ffhq", device)

### Best Buddies

In [ ]:
match_scores,_ = torch.max(table,1)

In [ ]:
n = 5
_,gan_matches1 = torch.topk(table,k=1,dim=1)
_,discr_matches1 = torch.topk(table,k=n, dim=0)


In [ ]:
gan_matches = torch.argmax(table,1)
dino_matches = torch.argmax(table,0)

In [ ]:
perfect_matches = []
perfect_match_scores1= []
discr_perfect_matches1 = []

num_kmatches = 0 
for i in range(table.shape[0]):
    gan_match = gan_matches1[i].item()
    discr_matches = discr_matches1[:, gan_match]
    
    for unit in discr_matches:
        if unit == i:
            num_kmatches += 1
            perfect_matches.append(i)
            discr_perfect_matches1.append(gan_match)
            perfect_match_scores1.append(table[i, gan_match])
            break
print(len(perfect_matches))

### Sort units according to scores

In [ ]:
gan_match_units = [match for _,match in sorted(zip(perfect_match_scores1, perfect_matches), reverse = True)]

In [ ]:
scores = sorted(perfect_match_scores1, reverse = True)
scores = [score.item() for score in scores]

## Visualize Matches over Sample Images

In [ ]:
batch_size = 1
epochs = 1
classidx = None
z_dataset, c_dataset = dataset.create_dataset(gan, "stylegan3", batch_size, epochs, classidx, device)

In [ ]:
init = gan(z_dataset,c_dataset,1)
init = (init.detach().cpu()+1)/2
init = torch.permute(init[0], (1,2,0))
plt.imshow(init)
init = (init-torch.min(init))/(torch.max(init)-torch.min(init))
init = np.uint8(init*255)

In [ ]:
gan_match_units.sort()

In [ ]:
#get number of units per each layer 
ganlayers, discrlayers = layers.get_layers(gan,gan_layers, discr, discr_layers,"stylegan3", "dino", device)

In [ ]:
visualize_pairwisematch.viz_matches(table, gan, discr, (z_dataset,c_dataset), ganlayers, discrlayers, gan_stats, discr_stats, "stylegan3", "dino", gan_match_units, scores )